In [ ]:
import requests

# Define the URL of the FastAPI endpoint
url = "http://127.0.0.1:8000/register-device"

# Define the schemas for different device types
schemas = {
    "gps": {
        "latitude": "float",
        "longitude": "float",
        "timestamp": "string"
    },
    "accel": {
        "acceleration_x": "float",
        "acceleration_y": "float",
        "acceleration_z": "float",
        "timestamp": "string"
    },
    "wind": {
        "speed": "float",
        "direction": "float",
        "timestamp": "string"
    },
    "player_temperature": {
        "temperature": "float",
        "timestamp": "string"
    },
    "player_heart_rate": {
        "heart_rate": "int",
        "timestamp": "string"
    }
}

# Number of devices to register for each type
num_devices = 10

# Loop through each device type and create multiple devices
for device_type, schema in schemas.items():
    for i in range(2, num_devices + 2):  # Starts at 2, ends at 11 (10 devices)
        device_name = f"{device_type}_{i}"
        payload = {
            "device_name": device_name,
            "schema": schema
        }
        
        # Send POST request to register the device
        response = requests.post(url, json=payload)
        
        # Print the status code and response for each device
        print(f"Registering device: {device_name}")
        print(f"Status Code: {response.status_code}")
        try:
            print(f"Response JSON: {response.json()}\n")
        except Exception as e:
            print(f"Error parsing response for {device_name}: {e}\n")


In [12]:
import asyncio
import websockets
import json
import random
import time
from datetime import datetime

# "Magic" Variables
WEBSOCKET_ENDPOINT_TEMPLATE = "ws://localhost:8000/send-stream/{device_id}/{run_id}"
DURATION_SECONDS = 500  # Total duration to send data
INTERVAL_SECONDS = 0.5  # Interval between data sends
OUT_OF_BOUNDS_INTERVAL = 100  # Every 100 messages, send an out-of-bounds message

# GPS-related boundaries and step sizes
PITCH_LAT_MIN = 40.0
PITCH_LAT_MAX = 40.0009
PITCH_LON_MIN = -75.0012
PITCH_LON_MAX = -75.0
STEP_SIZE_LAT = 0.00001  # Latitude step size for random walk (~1 meter)
STEP_SIZE_LON = 0.000015  # Longitude step size for random walk (~1 meter)

# Heart rate boundaries
HR_MIN = 70  # Minimum normal heart rate
HR_MAX = 140  # Maximum normal heart rate
HR_OUT_OF_BOUNDS_LOW = (30, 60)  # Out-of-bounds low range
HR_OUT_OF_BOUNDS_HIGH = (160, 200)  # Out-of-bounds high range

# Temperature boundaries
TEMP_MIN = 36.0  # Minimum normal temperature
TEMP_MAX = 39.0  # Maximum normal temperature
TEMP_OUT_OF_BOUNDS_LOW = (34.0, 35.5)  # Out-of-bounds low range
TEMP_OUT_OF_BOUNDS_HIGH = (39.5, 42.0)  # Out-of-bounds high range

def random_walk(current_lat, current_lon):
    """Perform a random walk within the boundaries of the football pitch."""
    new_lat = current_lat + random.uniform(-STEP_SIZE_LAT, STEP_SIZE_LAT)
    new_lon = current_lon + random.uniform(-STEP_SIZE_LON, STEP_SIZE_LON)
    new_lat = max(min(new_lat, PITCH_LAT_MAX), PITCH_LAT_MIN)
    new_lon = max(min(new_lon, PITCH_LON_MAX), PITCH_LON_MIN)
    return new_lat, new_lon

def generate_heart_rate():
    """Generate a random heart rate between HR_MIN and HR_MAX."""
    return random.randint(HR_MIN, HR_MAX)

def generate_temperature():
    """Generate a random temperature between TEMP_MIN and TEMP_MAX."""
    return round(random.uniform(TEMP_MIN, TEMP_MAX), 1)

async def send_synthetic_data(device_id, run_id, schema_type):
    websocket_endpoint = WEBSOCKET_ENDPOINT_TEMPLATE.format(device_id=device_id, run_id=run_id)

    async with websockets.connect(websocket_endpoint) as websocket:
        start_time = time.time()
        end_time = start_time + DURATION_SECONDS
        current_lat = (PITCH_LAT_MIN + PITCH_LAT_MAX) / 2
        current_lon = (PITCH_LON_MIN + PITCH_LON_MAX) / 2
        message_count = 0

        while time.time() < end_time:
            if schema_type == "gps":
                if message_count % OUT_OF_BOUNDS_INTERVAL == 0 and message_count > 0:
                    out_of_bounds_lat = PITCH_LAT_MAX + random.uniform(0.0001, 0.0005)
                    out_of_bounds_lon = PITCH_LON_MAX + random.uniform(0.0001, 0.0005)
                    data = {
                        "latitude": round(out_of_bounds_lat, 6),
                        "longitude": round(out_of_bounds_lon, 6),
                        "timestamp": datetime.utcnow().isoformat()
                    }
                else:
                    current_lat, current_lon = random_walk(current_lat, current_lon)
                    data = {
                        "latitude": round(current_lat, 6),
                        "longitude": round(current_lon, 6),
                        "timestamp": datetime.utcnow().isoformat()
                    }
            elif schema_type == "heart_rate":
                if message_count % OUT_OF_BOUNDS_INTERVAL == 0 and message_count > 0:
                    out_of_bounds_hr = random.choice(
                        [random.randint(*HR_OUT_OF_BOUNDS_LOW), random.randint(*HR_OUT_OF_BOUNDS_HIGH)]
                    )
                    data = {
                        "heart_rate": out_of_bounds_hr,
                        "timestamp": datetime.utcnow().isoformat()
                    }
                else:
                    heart_rate = generate_heart_rate()
                    data = {
                        "heart_rate": heart_rate,
                        "timestamp": datetime.utcnow().isoformat()
                    }
            elif schema_type == "temperature":
                if message_count % OUT_OF_BOUNDS_INTERVAL == 0 and message_count > 0:
                    out_of_bounds_temp = random.choice(
                        [round(random.uniform(*TEMP_OUT_OF_BOUNDS_LOW), 1), round(random.uniform(*TEMP_OUT_OF_BOUNDS_HIGH), 1)]
                    )
                    data = {
                        "temperature": out_of_bounds_temp,
                        "timestamp": datetime.utcnow().isoformat()
                    }
                else:
                    temperature = generate_temperature()
                    data = {
                        "temperature": temperature,
                        "timestamp": datetime.utcnow().isoformat()
                    }

            data_str = json.dumps(data)
            await websocket.send(data_str)
            print(f"Sent data from {device_id} (run {run_id}): {data_str}")

            message_count += 1
            await asyncio.sleep(INTERVAL_SECONDS)

async def main():
    run_id = "run_001"
    
    # Generate tasks for 10 devices of each type
    tasks = [
        *[send_synthetic_data(f"gps_{i}", run_id, "gps") for i in range(1, 4)],
        *[send_synthetic_data(f"player_heart_rate_{i}", run_id, "heart_rate") for i in range(1, 4)],
        *[send_synthetic_data(f"player_temperature_{i}", run_id, "temperature") for i in range(1, 4)]
    ]
    
    await asyncio.gather(*tasks)

# Run the main function
await main()


Sent data from gps_2 (run run_001): {"latitude": 40.000448, "longitude": -75.000588, "timestamp": "2024-12-10T12:55:03.487836"}
Sent data from gps_3 (run run_001): {"latitude": 40.000454, "longitude": -75.00061, "timestamp": "2024-12-10T12:55:03.940682"}
Sent data from player_heart_rate_3 (run run_001): {"heart_rate": 72, "timestamp": "2024-12-10T12:55:03.941937"}
Sent data from gps_1 (run run_001): {"latitude": 40.000459, "longitude": -75.000597, "timestamp": "2024-12-10T12:55:03.941937"}
Sent data from player_temperature_3 (run run_001): {"temperature": 37.9, "timestamp": "2024-12-10T12:55:03.942604"}
Sent data from gps_2 (run run_001): {"latitude": 40.000443, "longitude": -75.000593, "timestamp": "2024-12-10T12:55:03.998207"}
Sent data from player_heart_rate_1 (run run_001): {"heart_rate": 98, "timestamp": "2024-12-10T12:55:04.177049"}
Sent data from player_temperature_1 (run run_001): {"temperature": 38.3, "timestamp": "2024-12-10T12:55:04.178034"}
Sent data from gps_3 (run run_001

CancelledError: 